# Step 1: Extract from OSM
The purpose of this script is to show how to go from a .osm.pbf file to a network format that GOSTnets will accept. 
Additional modifications in this script include: clipping the roads to an input polygon (here, rek2.shp), and filtering the roads DataFrame by a list of accepted road types, as tagged in OSM (see accepted_road_types list). 

In [ ]:
import geopandas as gpd
import os

In [ ]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

Import the GOSTnets library as gn.

In [ ]:
import GOSTnets as gn

The load_osm module is an optional GOSTNets submodule that is used to import OSM data. Make sure gdal, geopy, and boltons are installed first as it has additional dependencies (look in GOSTnets repo main README file if you need additional help with this). Then import the submodule.

In [ ]:
from GOSTnets.load_osm import OSM_to_network

Define filepaths.

In [ ]:
pth = "./"  # change this path to your working folder
fil = r"iceland-latest.osm.pbf"  # download this file from geofabrik: http://download.geofabrik.de/europe/iceland.html.

# be sure to place the .osm.pbf file in the 'tutorial data' folder.

f = os.path.join(pth, "tutorial_data", fil)

In [ ]:
f

GOSTNets creates a special 'OSM_to_network' object. This object gets initialized with both a copy of the OSM file itself and the roads extracted from the OSM file in a GeoPandas DataFrame. This DataFrame is a property of the object called 'roads_raw' and is the starting point for our network.

In [ ]:
iceland = OSM_to_network(f)

In [ ]:
?iceland

In [ ]:
iceland.roads_raw.head()

In [ ]:
# show the different highway types and counts
iceland.roads_raw.infra_type.value_counts()

We define a list of the types of roads from the above that we consider acceptable for our road network

In [ ]:
accepted_road_types = [
    "residential",
    "unclassified",
    "track",
    "service",
    "tertiary",
    "road",
    "secondary",
    "primary",
    "trunk",
    "primary_link",
    "trunk_link",
    "tertiary_link",
    "secondary_link",
]

We call the filterRoads method and pass it a list of acceptable road types

In [ ]:
iceland.filterRoads(acceptedRoads=accepted_road_types)

In [ ]:
iceland.roads_raw.infra_type.value_counts()

Here, we import a shapefile for our area of interest. We want to make a map of Reykjavik, Iceland. We clip the extent of the country file to just the area around the capital (rek2.shp) below using standard GeoPandas functions:

In [ ]:
clip_shp = gpd.read_file(os.path.join(pth, "tutorial_data", "rek2.shp"))
clip_shp = clip_shp.to_crs({"init": "epsg:4326"})
clip_shp_obj = clip_shp.geometry.iloc[0]

In [ ]:
# this is a geopandas dataframe
clip_shp.plot()

In [ ]:
# this is the geometry object (shapely) from clip_shp
clip_shp_obj

We check to see everything lines up by running intersect and counting the True / False returns. The count of the True values are the number of roads that intersect the AOI

In [ ]:
# intersects is a Shapely function that returns True if the boundary or interior of the object intersect in any way with those of the other
iceland.roads_raw.geometry.intersects(clip_shp_obj).value_counts()

Remove any roads that don't intersect the capital area, by modifying the roads_raw from the 'OSM_to_network' object 

In [ ]:
iceland.roads_raw = iceland.roads_raw.loc[
    iceland.roads_raw.geometry.intersects(clip_shp_obj) == True  # noqa: E712
]

Having narrowed the DataFrame to the roads we are interested in, we generate the RoadsGPD object, which is stored as a property of the 'OSM_to_network' object. The RoadsGPD object is a GeoDataFrame that further processes the roads. This includes splitting the edges where intersections occur, adding unique edge IDs, and adding to/from columns to the GeoDataFrame.

In [ ]:
iceland.generateRoadsGDF(verbose=False)

In [ ]:
# view the first 5 rows of the newly created roads GeoDataFrame
iceland.roadsGPD.head()

With the initialReadIn() function, we transform this to a graph object

In [ ]:
iceland.initialReadIn()

We save this graph object down to file using gn.save(). The save function produces three outputs: a node GeoDataFrame as a CSV, an edge GeoDataFrame as a CSV, and a graph object saved as a pickle.

In [ ]:
gn.save?

In [ ]:
gn.save(iceland.network, "iceland_unclean", os.path.join(pth, "tutorial_outputs"))

Move on to Step 2 to see how we clean up this rough network.